In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = [
    ("hello", "வணக்கம்"),
    ("how are you", "நீங்கள் எப்படி இருக்கிறீர்கள்"),
    ("thank you", "நன்றி"),
    ("good morning", "காலை வணக்கம்"),
    ("good night", "இனிய இரவு"),
    ("what is your name", "உங்கள் பெயர் என்ன"),
    ("i am fine", "நான் நன்றாக இருக்கிறேன்"),
    ("where are you", "நீங்கள் எங்கே இருக்கிறீர்கள்"),
    ("nice to meet you", "உங்களை சந்தித்ததில் மகிழ்ச்சி"),
    ("i love you", "நான் உன்னை நேசிக்கிறேன்"),
    ("please sit down", "தயவுசெய்து உட்காருங்கள்"),
    ("come here", "இங்கே வாருங்கள்"),
    ("go there", "அங்கே செல்லுங்கள்"),
    ("do you speak english", "நீங்கள் ஆங்கிலம் பேசுகிறீர்களா"),
    ("i don't understand", "எனக்குப் புரியவில்லை"),
    ("can you help me", "நீங்கள் எனக்கு உதவ முடியுமா"),
    ("i am hungry", "எனக்கு பசிக்கிறது"),
    ("this is my friend", "இவர் என் நண்பர்"),
    ("where is the bathroom", "கழிப்பறை எங்கே உள்ளது"),
    ("i am learning tamil", "நான் தமிழைக் கற்றுக்கொண்டு இருக்கிறேன்")
]

def build_vocab(sentences):
    vocab = {"<pad>": 0, "<sos>": 1, "<eos>": 2}
    idx = 3
    for sentence in sentences:
        for word in sentence.split():
            if word not in vocab:
                vocab[word] = idx
                idx += 1
    return vocab

src_vocab = build_vocab([src for src, _ in data])
trg_vocab = build_vocab([trg for _, trg in data])
inv_trg_vocab = {v: k for k, v in trg_vocab.items()}

def encode(sentence, vocab):
    return [vocab["<sos>"]] + [vocab[word] for word in sentence.split()] + [vocab["<eos>"]]

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.gru(embedded)
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim)
        self.fc_out = nn.Linear(hid_dim, output_dim)

    def forward(self, input, hidden):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        output, hidden = self.gru(embedded, hidden)
        prediction = self.fc_out(output.squeeze(0))
        return prediction.squeeze(0), hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.fc_out.out_features
        outputs = torch.zeros(trg_len, trg_vocab_size).to(device)
        hidden = self.encoder(src)
        input = trg[0].unsqueeze(0)
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[t] = output
            top1 = output.argmax(0)
            input = trg[t].unsqueeze(0) if random.random() < teacher_forcing_ratio else top1.unsqueeze(0)
        return outputs

INPUT_DIM = len(src_vocab)
OUTPUT_DIM = len(trg_vocab)
EMB_DIM = 64
HID_DIM = 128

enc = Encoder(INPUT_DIM, EMB_DIM, HID_DIM).to(device)
dec = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM).to(device)
model = Seq2Seq(enc, dec).to(device)

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

def train(n_epochs=50):
    model.train()
    for epoch in range(n_epochs):
        total_loss = 0
        for src_sent, trg_sent in data:
            src_tensor = torch.tensor(encode(src_sent, src_vocab), dtype=torch.long).unsqueeze(1).to(device)
            trg_tensor = torch.tensor(encode(trg_sent, trg_vocab), dtype=torch.long).to(device)
            optimizer.zero_grad()
            output = model(src_tensor, trg_tensor)
            output_dim = output.shape[-1]
            loss = criterion(output[1:].view(-1, output_dim), trg_tensor[1:])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}: Loss = {total_loss/len(data):.4f}")

def translate(sentence):
    model.eval()
    with torch.no_grad():
        src_tensor = torch.tensor(encode(sentence, src_vocab), dtype=torch.long).unsqueeze(1).to(device)
        hidden = model.encoder(src_tensor)
        input = torch.tensor([trg_vocab["<sos>"]], dtype=torch.long).to(device)
        result = []
        for _ in range(20):
            output, hidden = model.decoder(input, hidden)
            top1 = output.argmax(0).item()
            if top1 == trg_vocab["<eos>"]:
                break
            result.append(inv_trg_vocab[top1])
            input = torch.tensor([top1], dtype=torch.long).to(device)
        return " ".join(result)

train()
print(translate("hello"))
print(translate("how are you"))
print(translate("i am fine"))
print(translate("where is the bathroom"))


Epoch 10: Loss = 0.9247
Epoch 20: Loss = 0.1225
Epoch 30: Loss = 0.0432
Epoch 40: Loss = 0.0228
Epoch 50: Loss = 0.0142
வணக்கம்
நீங்கள் எப்படி இருக்கிறீர்கள்
நான் நன்றாக இருக்கிறேன்
கழிப்பறை எங்கே உள்ளது
